In [1]:
import os
import sys
import warnings

import numpy as np
from sympy import Symbol, Function, Number

import modulus.sym
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
    PointwiseConstraint,
)
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.geometry.primitives_2d import Rectangle
from modulus.sym.key import Key
from modulus.sym.eq.pdes.wave_equation import WaveEquation
from modulus.sym.eq.pde import PDE
from modulus.sym.utils.io.plotter import ValidatorPlotter


# Read in npz files generated using finite difference simulator Devito
def read_wf_data(time, dLen):
    file_path = "Training_data"
    
    if not os.path.exists(to_absolute_path(file_path)):
        warnings.warn(
            f"Directory {file_path} does not exist. Cannot continue. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
        )
        sys.exit()
    wf_filename = to_absolute_path(f"Training_data/wf_{int(time):04d}ms.npz")
    wave = np.load(wf_filename)["arr_0"].astype(np.float32)
    mesh_y, mesh_x = np.meshgrid(
        np.linspace(0, dLen, wave.shape[0]),
        np.linspace(0, dLen, wave.shape[1]),
        indexing="ij",
    )
    invar = {}
    invar["x"] = np.expand_dims(mesh_y.astype(np.float32).flatten(), axis=-1)
    invar["y"] = np.expand_dims(mesh_x.astype(np.float32).flatten(), axis=-1)
    invar["t"] = np.full_like(invar["x"], time * 0.001)
    outvar = {}
    outvar["u"] = np.expand_dims(wave.flatten(), axis=-1)
    return invar, outvar


# define open boundary conditions
class OpenBoundary(PDE):
    """
    Open boundary condition for wave problems
    Ref: http://hplgit.github.io/wavebc/doc/pub/._wavebc_cyborg002.html

    Parameters
    ==========
    u : str
        The dependent variable.
    c : float, Sympy Symbol/Expr, str
        Wave speed coefficient. If `c` is a str then it is
        converted to Sympy Function of form 'c(x,y,z,t)'.
        If 'c' is a Sympy Symbol or Expression then this
        is substituted into the equation.
    dim : int
        Dimension of the wave equation (1, 2, or 3). Default is 2.
    time : bool
        If time-dependent equations or not. Default is True.
    """

    name = "OpenBoundary"

    def __init__(self, u="u", c="c", dim=3, time=True):
        # set params
        self.u = u
        self.dim = dim
        self.time = time

        # coordinates
        x, y, z = Symbol("x"), Symbol("y"), Symbol("z")

        # normal
        normal_x, normal_y, normal_z = (
            Symbol("normal_x"),
            Symbol("normal_y"),
            Symbol("normal_z"),
        )

        # time
        t = Symbol("t")

        # make input variables
        input_variables = {"x": x, "y": y, "z": z, "t": t}
        if self.dim == 1:
            input_variables.pop("y")
            input_variables.pop("z")
        elif self.dim == 2:
            input_variables.pop("z")
        if not self.time:
            input_variables.pop("t")

        # Scalar function
        assert type(u) == str, "u needs to be string"
        u = Function(u)(*input_variables)

        # wave speed coefficient
        if type(c) is str:
            c = Function(c)(*input_variables)
        elif type(c) in [float, int]:
            c = Number(c)

        # set equations
        self.equations = {}
        self.equations["open_boundary"] = (
            u.diff(t)
            + normal_x * c * u.diff(x)
            + normal_y * c * u.diff(y)
            + normal_z * c * u.diff(z)
        )


class WavePlotter(ValidatorPlotter):
    "Define custom validator plotting class"

    def __call__(self, invar, true_outvar, pred_outvar):
        # only plot x,y dimensions
        invar = {k: v for k, v in invar.items() if k in ["x", "y"]}
        fs = super().__call__(invar, true_outvar, pred_outvar)
        return fs



In [2]:
file_path = "Training_data"
print(to_absolute_path(file_path))

/home/mitch/PythonProjects/explosics-learn/tutorials/wave2d/Training_data


In [3]:
from modulus.sym.hydra.utils import compose
from modulus.sym.hydra import to_yaml
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/home/mitch/miniconda3/envs/phd/lib/python3.9/site-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 40000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: 1000
  rec_inference_freq: 1000
  rec_monitor_freq: 1000
  rec_constraint_freq: 2000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk
summary_histograms: true
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arch:
  fully_connected:
    arch_type: fully_connected
    input_keys: ???
    ou

In [4]:
import logging
logging.getLogger().addHandler(logging.StreamHandler())

In [7]:
wave_net

FullyConnectedArch(
  (_impl): FullyConnectedArchCore(
    (layers): ModuleList(
      (0): FCLayer(
        (linear): WeightNormLinear(in_features=3, out_features=128, bias=True)
      )
      (1-5): 5 x FCLayer(
        (linear): WeightNormLinear(in_features=128, out_features=128, bias=True)
      )
    )
    (final_layer): FCLayer(
      (activation_fn): Identity()
      (linear): Linear(in_features=128, out_features=1, bias=True)
    )
  )
)

In [9]:
from torchinfo import summary
summary(wave_net, input_size=(1, 2))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []

In [5]:


"""
2d acoustic wave propagation at a domain of 2kmx2km, with a single Ricker source at the middle of the 2D domain
"""

# override defaults
cfg.arch.fully_connected.layer_size = 128

# define PDEs
we = WaveEquation(u="u", c="c", dim=2, time=True)
ob = OpenBoundary(u="u", c="c", dim=2, time=True)

# define networks and nodes
wave_net = instantiate_arch(
    input_keys=[Key("x"), Key("y"), Key("t")],
    output_keys=[Key("u")],
    cfg=cfg.arch.fully_connected,
)
speed_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("c")],
    cfg=cfg.arch.fully_connected,
)
nodes = (
    we.make_nodes(detach_names=["c"])
    + ob.make_nodes(detach_names=["c"])
    + [
        wave_net.make_node(name="wave_network"),
        speed_net.make_node(name="speed_network"),
    ]
)

# define geometry
dLen = 2  # km
rec = Rectangle((0, 0), (dLen, dLen))

# define sympy domain variables
x, y, t = Symbol("x"), Symbol("y"), Symbol("t")

# define time range
time_length = 1
time_range = {t: (0.15, time_length)}

# define target velocity model
# 2.0 km/s at the bottom and 1.0 km/s at the top using tanh function
mesh_x, mesh_y = np.meshgrid(
    np.linspace(0, 2, 512), np.linspace(0, 2, 512), indexing="ij"
)
wave_speed_invar = {}
wave_speed_invar["x"] = np.expand_dims(mesh_x.flatten(), axis=-1)
wave_speed_invar["y"] = np.expand_dims(mesh_y.flatten(), axis=-1)
wave_speed_outvar = {}
wave_speed_outvar["c"] = np.tanh(80 * (wave_speed_invar["y"] - 1.0)) / 2 + 1.5

# make domain
domain = Domain()

# add velocity constraint
velocity = PointwiseConstraint.from_numpy(
    nodes=nodes, invar=wave_speed_invar, outvar=wave_speed_outvar, batch_size=1024
)
domain.add_constraint(velocity, "Velocity")

# add initial timesteps constraints
batch_size = 1024
for i, ms in enumerate(np.linspace(150, 300, 4)):
    timestep_invar, timestep_outvar = read_wf_data(ms, dLen)
    lambda_weighting = {}
    lambda_weighting["u"] = np.full_like(timestep_invar["x"], 10.0 / batch_size)
    timestep = PointwiseConstraint.from_numpy(
        nodes,
        timestep_invar,
        timestep_outvar,
        batch_size,
        lambda_weighting=lambda_weighting,
    )
    domain.add_constraint(timestep, f"BC{i:04d}")

# add interior constraint
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"wave_equation": 0},
    batch_size=4096,
    bounds={x: (0, dLen), y: (0, dLen)},
    lambda_weighting={"wave_equation": 0.0001},
    parameterization=time_range,
)
domain.add_constraint(interior, "Interior")

# add open boundary constraint
edges = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"open_boundary": 0},
    batch_size=1024,
    lambda_weighting={"open_boundary": 0.01 * time_length},
    parameterization=time_range,
)
domain.add_constraint(edges, "Edges")

# add validators
for i, ms in enumerate(np.linspace(350, 950, 13)):
    val_invar, val_true_outvar = read_wf_data(ms, dLen)
    validator = PointwiseValidator(
        nodes=nodes,
        invar=val_invar,
        true_outvar=val_true_outvar,
        batch_size=1024,
        plotter=WavePlotter(),
    )
    domain.add_validator(validator, f"VAL_{i:04d}")
validator = PointwiseValidator(
    nodes=nodes,
    invar=wave_speed_invar,
    true_outvar=wave_speed_outvar,
    batch_size=1024,
    plotter=WavePlotter(),
)
domain.add_validator(validator, "Velocity")

slv = Solver(cfg, domain)

slv.solve()

Arch Node: wave_network has been converted to a FuncArch node.
Arch Node: wave_network has been converted to a FuncArch node.
Installed PyTorch version 2.2.0+cu121 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
attempting to restore from: /home/mitch/PythonProjects/explosics-learn/tutorials/wave2d/outputs
optimizer checkpoint not found
model speed_network.0.pth not found
model wave_network.0.pth not found
[step:          0] saved constraint results to outputs
[step:          0] record constraint batch time:  7.058e-02s
Loaded backend module://matplotlib_inline.backend_inline version unknown.
Loaded backend module://matplotlib_inline.backend_inline version unknown.
findfont: Matching sans\-serif:style=normal:variant=normal:weight=normal:stretch=normal:size=10.0.
findfont: score(FontEntry(fname='/home/mitch/miniconda3/envs/phd/lib/python3.9/site-packages/matplotlib/mpl-data/fonts/ttf/STIXNonUni.ttf', name='STIXNonUnicode', style='normal', varian

KeyboardInterrupt: 